In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Hit:13 http://ppa.launchpad.net

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-05-16 23:16:42--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.38MB/s    in 0.2s    

2021-05-16 23:16:43 (4.38 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL_AWS_mobile_electronic").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Setup config file location
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/My Drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from config import aws_rds_endpoint
from config import aws_rds_password
# Configuration for RDS instance
jdbc_url = f"jdbc:postgresql://{aws_rds_endpoint}:5432/big_data_db"
mode_1="append"
config_1 = {"user":"postgres",
          "password": aws_rds_password,
          "driver":"org.postgresql.Driver"}
mode_2="overwrite"
config_2 = {"user":"postgres",
          "password": aws_rds_password,
          "truncate": "true",
          "driver":"org.postgresql.Driver"}

In [7]:
from pyspark import SparkFiles
#load software reviews data from s3.amazonaws.com
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

mobelec_df = spark.read.option("header","true").csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"), inferSchema=True, dateFormat="yyyy-MM-dd",sep="\t")
mobelec_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

# **Drop duplicates and incomplete rows**

In [8]:
mobelec_df = mobelec_df.drop("marketplace","product_category","verified_purchase","review_headline","review_body")
print(mobelec_df.count())
mobelec_df = mobelec_df.dropna()
print(mobelec_df.count())
mobelec_df = mobelec_df.dropDuplicates()
print(mobelec_df.count())

104975
104973
104973


# **Examine the schema**

In [9]:
mobelec_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: string (nullable = true)



# **Transfor Dataframe type**

In [10]:
import datetime
mobelec_df = mobelec_df.withColumn("review_date",mobelec_df["review_date"].cast("date"))
mobelec_df.show(5)

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   11491447|R2OCDOIHJVF0EJ|B00MJCDPM2|     754171872|Sentey LS-4460 B-...|          5|            0|          0|   N| 2015-08-25|
|    2737267|R18R2FSKAOINPA|B00HTXPBV4|     854337141|Lychee® Fish Find...|          5|            1|          1|   N| 2015-08-25|
|   30554689|R3H5NR66VPQYF4|B00RCO24O2|     915831598|Aweek® 5M Coaxial...|          5|            0|          0|   N| 2015-08-24|
|   44131190|R10G3P0YNL7O7G|B00AOH0MKM|     942607823|Pyle PylePro PDWR...|          5|            1|          1|   N| 2015-08-23|
|     658913| R4SFDXU19AFFA|B00IWVE0Q4|     916527392|LIMTECH GENUINE 7...|        

In [11]:
mobelec_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: date (nullable = true)



# **Create Dataframe for review_id_table**

In [12]:
review_id_df = mobelec_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2OCDOIHJVF0EJ|   11491447|B00MJCDPM2|     754171872| 2015-08-25|
|R18R2FSKAOINPA|    2737267|B00HTXPBV4|     854337141| 2015-08-25|
|R3H5NR66VPQYF4|   30554689|B00RCO24O2|     915831598| 2015-08-24|
|R10G3P0YNL7O7G|   44131190|B00AOH0MKM|     942607823| 2015-08-23|
| R4SFDXU19AFFA|     658913|B00IWVE0Q4|     916527392| 2015-08-17|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [13]:
# Drop duplicated review existed in sql database
review_id_df_pre = spark.read.jdbc(url=jdbc_url,table="review_id_table", properties=config_1)
review_id_df = review_id_df.join(review_id_df_pre, ["review_id"], "leftanti")
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10CICGOU7CUI8|   23140361|B000G2JGU2|      55661033| 2007-09-19|
|R10K7G8PMZHWY5|   20460922|B003TZ8YRE|     597744426| 2011-05-02|
|R10TLNH0GLYEID|   12140794|B00M8X28FM|     663332731| 2014-12-10|
|R11D7R3VBRYB26|    8234778|B00E3QF8C2|     580429325| 2015-03-08|
|R1209VB02CM4ZD|   27382433|B007WO4ENG|     567489042| 2013-12-21|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [14]:
print(review_id_df.count())
review_id_df = review_id_df.dropDuplicates(["review_id"])
print(review_id_df.count())

104973
104973


# **Create Dataframe for products_table**

In [15]:
products_df = mobelec_df.select(["product_id", "product_title"])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00MJCDPM2|Sentey LS-4460 B-...|
|B00HTXPBV4|Lychee® Fish Find...|
|B00RCO24O2|Aweek® 5M Coaxial...|
|B00AOH0MKM|Pyle PylePro PDWR...|
|B00IWVE0Q4|LIMTECH GENUINE 7...|
+----------+--------------------+
only showing top 5 rows



In [16]:
# Drop duplicated review existed in sql database
products_df_pre = spark.read.jdbc(url=jdbc_url,table="products", properties=config_1)
products_df = products_df.join(products_df_pre, ["product_id"], "leftanti")
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0006BFQUM|Eminence American...|
|B0006BFQUM|Eminence American...|
|B0009YV3U8|Pyle PLCAP60HB 6 ...|
|B000A50URS|Panasonic Digital...|
|B000A50URS|Panasonic Digital...|
+----------+--------------------+
only showing top 5 rows



In [17]:
print(products_df.count())
products_df = products_df.dropDuplicates(["product_id"])
print(products_df.count())

104973
25800


# **Create Dataframe for customers_table**

In [18]:
customers_df_new = mobelec_df.select(["customer_id"])
customers_df_new = customers_df_new.groupBy("customer_id").count()
customers_df_new.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    1397686|    1|
|    1018724|    1|
|   36548940|    1|
|   22289410|    1|
|   24879973|    1|
+-----------+-----+
only showing top 5 rows



In [19]:
# Creat a copy of sql database
customers_df_pre = spark.read.jdbc(url=jdbc_url,table="customers", properties=config_1)
schema = customers_df_pre.schema
customers_df_pre_pd = customers_df_pre.toPandas()
_customers_df_pre = spark.createDataFrame(customers_df_pre_pd,schema=schema)
del customers_df_pre_pd

In [20]:
# Get the total counts of each customer
if _customers_df_pre.count():
  customers_df=customers_df_new.join(_customers_df_pre,on=["customer_id"],how="outer")
  customers_df=customers_df.fillna(0)
  customers_df=customers_df.withColumn('total', sum(customers_df[col] for col in ["count", "customer_count"]))
  customers_df = customers_df.drop("count", "customer_count")
  customers_df = customers_df.withColumnRenamed("total", "customer_count")
  customers_df.show(5)
else:
  customers_df = customers_df_new.withColumnRenamed("count", "customer_count")
  customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    1397686|             1|
|    1018724|             1|
|   36548940|             1|
|   22289410|             1|
|   24879973|             1|
+-----------+--------------+
only showing top 5 rows



In [21]:
print(customers_df.count())

101154


# **Create Dataframe for vine_table**

In [22]:
vine_df = mobelec_df.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2OCDOIHJVF0EJ|          5|            0|          0|   N|
|R18R2FSKAOINPA|          5|            1|          1|   N|
|R3H5NR66VPQYF4|          5|            0|          0|   N|
|R10G3P0YNL7O7G|          5|            1|          1|   N|
| R4SFDXU19AFFA|          1|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [23]:
# Drop duplicated review existed in sql database
vine_df_pre = spark.read.jdbc(url=jdbc_url,table="vine_table", properties=config_1)
vine_df = vine_df.join(vine_df_pre, ["review_id"], "leftanti")
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10CICGOU7CUI8|          5|            1|          1|   N|
|R10K7G8PMZHWY5|          4|            5|          5|   N|
|R10TLNH0GLYEID|          1|            1|          2|   N|
|R11D7R3VBRYB26|          5|            0|          0|   N|
|R1209VB02CM4ZD|          5|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [24]:
print(vine_df.count())
vine_df = vine_df.dropDuplicates(["review_id"])
print(vine_df.count())

104973
104973


# **Write DataFrame to RDS**

In [25]:
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode_1, properties=config_1)

In [26]:
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode_1, properties=config_1)

In [27]:
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode_2, properties=config_2)

In [28]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode_1, properties=config_1)